In [2]:
!pip install pyarrow

In [3]:
import pandas as pd
import pickle
import numpy as np
import re
import numpy as np



with open('/content/drive/MyDrive/full_data.pickle', 'rb') as f:
    full_data = pd.read_pickle(f)

filtered_data = full_data[full_data['date_filed'] >= full_data['date_start']]
# turn opinions_cited into the length of the list
filtered_data['opinions_cited'] = filtered_data['opinions_cited'].apply(lambda x: len(x))

def create_full_name(first_name, middle_name, last_name, suffix):
    name_parts = [first_name, middle_name, last_name, suffix]
    print(name_parts)
    # Filter out empty parts and join them with a space and remember about the NaNs and the ''s
    full_name = " ".join(
        [part for part in name_parts if part not in [np.nan, None, "None", ""]]
    )
    return full_name

filtered_data["full_name"] = filtered_data.apply(
    lambda x: create_full_name(
        x["name_first"], x["name_middle"], x["name_last"], x["name_suffix"]
    ),
    axis=1,
)

filtered_data = filtered_data.drop(columns=["name_first", "name_middle", "name_last", "name_suffix"])

def replace_nan_with_none(x):
    if isinstance(x, list):
        return [item if pd.notna(item) else None for item in x]
    return None if pd.isna(x) else x

columns_to_fix = [
    "date_dob",
    "political_party",
]

for col in columns_to_fix:
    filtered_data[col] = filtered_data[col].apply(replace_nan_with_none)

def convert_opinions_cited(x):
    if pd.isna(x):
        return None
    elif isinstance(x, int):
        return x
    else:
        return None

# Create a new DataFrame by explicitly copying filtered_data
filtered_data_copy = filtered_data.copy()

# make all of the sub_opinions into an excel file
filtered_data_copy['sub_opinions'].to_excel('sub_opinions.xlsx')


filtered_data_copy.loc[:, "opinions_cited"] = filtered_data_copy["opinions_cited"].apply(convert_opinions_cited)

# Convert "None" strings to None in opinions_text
filtered_data_copy['opinions_text'] = filtered_data_copy['opinions_text'].apply(lambda lst: [None if x == "None" else x for x in lst])

# Replace empty lists in opinions_text with None
filtered_data_copy['opinions_text'] = filtered_data_copy['opinions_text'].apply(lambda x: None if len(x) == 0 else x)

# convert opinions_text into utf-8
# Convert opinions_text elements to utf-8
filtered_data_copy["opinions_text"] = filtered_data_copy["opinions_text"].apply(
    lambda x: [item.encode("utf-8") if item is not None else None for item in x] if isinstance(x, list) else None
)

import re
import pandas as pd

def clean_string(text):
    if isinstance(text, str):
        # Remove newline characters and replace with a single space
        text = re.sub(r'\n+', ' ', text)
        # Remove any type of whitespace characters and replace with a single space
        text = re.sub(r'\s+', ' ', text)
        # Remove multiple consecutive backslashes
        text = re.sub(r'\\+', '', text)
        # Remove any remaining special characters using regular expressions
        text = re.sub(r'[^\w\s-]', '', text)
        # Remove leading/trailing whitespace
        text = text.strip()
        return text
    else:
        return text

def clean_list(lst):
    if isinstance(lst, list):
        return [clean_list(item) if isinstance(item, list) else clean_string(item) for item in lst]
    else:
        return clean_string(lst)

def clean_opinions_text(opinions_text):
    if isinstance(opinions_text, list):
        return [clean_string(text) for text in opinions_text]
    else:
        return clean_string(opinions_text)

#decode out of bytes

filtered_data_copy["opinions_text"] = filtered_data_copy["opinions_text"].apply(lambda lst: [item.decode("utf-8") if isinstance(item, bytes) else item for item in lst] if lst is not None else None)
# Apply clean_opinions_text to the "opinions_text" column
filtered_data_copy["opinions_text"] = filtered_data_copy["opinions_text"].apply(clean_opinions_text)


import pyarrow.feather as feather

# Save the cleaned DataFrame to a Feather file
feather.write_feather(filtered_data_copy, 'cleaned_data.feather')





['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Sonia', '', 'Sotomayor', '']
['Samuel', 'A.', 'Alito', 'jr']
['Samuel', 'A.', 'Alito', 'jr']
['Samuel', 'A.', 'Alito', 'jr']
['Samuel', 'A.', 'Alito', 'jr']
['Samuel', 'A.', 'Alito', 'jr']
['Samuel', 'A.', 'Alito', 'jr']
['